In [1]:
import os
import numpy as np 
import pandas as pd
from scipy import stats, signal, io
import mne
from mne import create_info, EpochsArray
from mne.time_frequency import tfr_morlet
from bids import BIDSLayout
import mne_bids
import scipy
#import settings
import json
#from coordinates_io import BIDS_coord
from sklearn import linear_model, metrics
import multiprocessing
from sklearn.model_selection import cross_val_score
from sklearn import ensemble, model_selection
from matplotlib import pyplot as plt
import matplotlib 

In [2]:
out_path = '/home/icn/Documents/raw_out/RF_32_4/'
BIDS_path = '/home/icn/Documents/BIDS/'

In [3]:
subject_exclude = ['002', '003', '011', '012', '015']
subject_tf_usage = ['000', '001', '004', '005', '006', '007', '008', '009', '010', '013', '014', '016']
#002 - DBS4011 - no locaization possible
#003 - DBS4020 - LEFT due to signal quality
#011 - DBS4037 - STN was not recorded
#012 - DBS4030 - GPi not STN 
#015 - DBS4065 - LEFT due to signal quality

In [4]:
subject_id = []
for patient_test in range(17):
    if patient_test < 10:
        subject_id.append(str('00') + str(patient_test))
    else:
        subject_id.append(str('0') + str(patient_test))


In [5]:
layout = BIDSLayout(BIDS_path)
tsv_files = layout.get(extension='tsv', return_type='filename')
vhdr_files = layout.get(extension='vhdr', return_type='filename')

In [6]:
subject_id_ = '000'
with open(out_path+'sub_'+subject_id_+'_patient_concat.json', 'r') as fp:
    dict_ch = json.load(fp)

In [8]:
dict_ch.keys()

dict_keys(['ECOG_RIGHT_0', 'ECOG_RIGHT_1', 'ECOG_RIGHT_2', 'ECOG_RIGHT_3', 'ECOG_RIGHT_4', 'ECOG_RIGHT_5', 'STN_RIGHT_0', 'STN_RIGHT_1', 'STN_RIGHT_2', 'STN_RIGHT_3'])

In [23]:
ECOG_ch = np.array([idx for idx, ch in enumerate(list(dict_ch.keys())) if ch.startswith('ECOG_') is True])

In [45]:
channels = list(dict_ch.keys())

In [38]:
dat_combine.shape

(48, 7570)

In [77]:
out_path

'/home/icn/Documents/raw_out/RF_32_4/'

In [78]:
contra_ = []; ipsi_ = []
for _, subject_id_ in enumerate(subject_id):
    if subject_id_ in subject_exclude:
        continue

    with open(out_path+'sub_'+subject_id_+'_patient_concat.json', 'r') as fp:
        dict_ch = json.load(fp)
    
    print(subject_id_)

    ECOG_ch = np.array([idx for idx, ch in enumerate(list(dict_ch.keys())) if ch.startswith('ECOG_') is True])
    channels = list(dict_ch.keys())
    start = 0
    for idx_ch,_ in enumerate(ECOG_ch):
        if start == 0:
            dat_combine = np.array(dict_ch[channels[ECOG_ch[idx_ch]]]['data'])
        else:
            dat_combine = np.concatenate((dat_combine, np.array(dict_ch[channels[ECOG_ch[idx_ch]]]['data'])), axis=0)

    
    mov_ = np.array(dict_ch[channels[ECOG_ch[idx_ch]]]['true_movements'])
    mov_ch = dict_ch[channels[ECOG_ch[idx_ch]]]['mov_ch']

    for mov_idx in range(2):
        if ('RIGHT' in mov_ch[mov_idx] and 'RIGHT' in channels[ECOG_ch[idx_ch]]) or \
           ('LEFT' in mov_ch[mov_idx] and 'LEFT' in channels[ECOG_ch[idx_ch]]): 
            ipsi_.append(np.mean(cross_val_score(linear_model.LinearRegression(), \
                                                 dat_combine.T, mov_[mov_idx,:], cv=5, scoring='r2', n_jobs=62)))

        if ('RIGHT' in mov_ch[mov_idx] and 'LEFT' in channels[ECOG_ch[idx_ch]]) or \
           ('LEFT' in mov_ch[mov_idx] and 'RIGHT' in channels[ECOG_ch[idx_ch]]): 
            contra_.append(np.mean(cross_val_score(linear_model.LinearRegression(), \
                                                    dat_combine.T, mov_[mov_idx,:], cv=5, scoring='r2', n_jobs=62)))



000
001


ValueError: Found input variables with inconsistent numbers of samples: [11670, 10960]

In [75]:
ipsi_

[0.12435731951266593,
 0.12435731951266593,
 0.12435731951266593,
 0.12435731951266593,
 0.12435731951266593,
 0.12435731951266593]

In [76]:
contra_

[0.4930855951776335,
 0.4930855951776335,
 0.4930855951776335,
 0.4930855951776335,
 0.4930855951776335,
 0.4930855951776335]

In [72]:
ECOG_ch

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59])

In [73]:
channels

['ECOG_RIGHT_0',
 'ECOG_RIGHT_1',
 'ECOG_RIGHT_2',
 'ECOG_RIGHT_3',
 'ECOG_RIGHT_4',
 'ECOG_RIGHT_5',
 'ECOG_RIGHT_6',
 'ECOG_RIGHT_7',
 'ECOG_RIGHT_8',
 'ECOG_RIGHT_9',
 'ECOG_RIGHT_10',
 'ECOG_RIGHT_11',
 'ECOG_RIGHT_12',
 'ECOG_RIGHT_13',
 'ECOG_RIGHT_14',
 'ECOG_RIGHT_15',
 'ECOG_RIGHT_16',
 'ECOG_RIGHT_17',
 'ECOG_RIGHT_18',
 'ECOG_RIGHT_19',
 'ECOG_RIGHT_20',
 'ECOG_RIGHT_21',
 'ECOG_RIGHT_22',
 'ECOG_RIGHT_23',
 'ECOG_RIGHT_24',
 'ECOG_RIGHT_25',
 'ECOG_RIGHT_26',
 'ECOG_RIGHT_27',
 'STN_RIGHT_0',
 'STN_RIGHT_1',
 'STN_RIGHT_2',
 'STN_RIGHT_3',
 'ECOG_LEFT_0',
 'ECOG_LEFT_1',
 'ECOG_LEFT_2',
 'ECOG_LEFT_3',
 'ECOG_LEFT_4',
 'ECOG_LEFT_5',
 'ECOG_LEFT_6',
 'ECOG_LEFT_7',
 'ECOG_LEFT_8',
 'ECOG_LEFT_9',
 'ECOG_LEFT_10',
 'ECOG_LEFT_11',
 'ECOG_LEFT_12',
 'ECOG_LEFT_13',
 'ECOG_LEFT_14',
 'ECOG_LEFT_15',
 'ECOG_LEFT_16',
 'ECOG_LEFT_17',
 'ECOG_LEFT_18',
 'ECOG_LEFT_19',
 'ECOG_LEFT_20',
 'ECOG_LEFT_21',
 'ECOG_LEFT_22',
 'ECOG_LEFT_23',
 'ECOG_LEFT_24',
 'ECOG_LEFT_25',
 'ECOG_LEF

In [54]:
mov_ = np.array(dict_ch['ECOG_RIGHT_0']['true_movements'])


In [55]:
mov_.shape

(2, 7570)